In [1]:
import numpy as np
import gym
import time
import math
import matplotlib.pyplot as plt 

In [2]:
env = gym.make("CartPole-v1")
print(env.action_space.n)

2


In [3]:
# Learning Rate: learning rate is associated with how big you take a leap
lr = 0.1

#Discount Factor
gamma = 0.95

#Amount of iterations we are going to run until we see our model is trained
epochs = 60000
total_time = 0
total_reward = 0
prev_reward = 0

Observation = [30, 30, 50, 50]
step_size = np.array([0.25, 0.25, 0.01, 0.1])

# epsilon is associated with how random you take an action.
epsilon = 1

#exploration is decaying and we will get to a state of full exploitation
epsilon_decay_value = 0.99995

In [4]:
#randomly initializing values in our q table
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape
print(q_table[0][0])

[[[0.77429742 0.08036428]
  [0.29981906 0.7422782 ]
  [0.2408664  0.8512125 ]
  ...
  [0.88078673 0.64962207]
  [0.00832906 0.00616098]
  [0.53874865 0.88548592]]

 [[0.75604789 0.51853906]
  [0.16041957 0.75481486]
  [0.00711429 0.24596698]
  ...
  [0.76752669 0.71817257]
  [0.57723556 0.6001075 ]
  [0.64530213 0.97621228]]

 [[0.62702025 0.9267439 ]
  [0.67162353 0.85966611]
  [0.4814359  0.94357726]
  ...
  [0.90839629 0.12777662]
  [0.31781654 0.9676655 ]
  [0.56642113 0.89659706]]

 ...

 [[0.75966149 0.48994113]
  [0.0892755  0.12262169]
  [0.86083793 0.72144704]
  ...
  [0.45768784 0.31438382]
  [0.10461402 0.00439902]
  [0.46653829 0.96732693]]

 [[0.67267929 0.45589636]
  [0.77600148 0.47846988]
  [0.25148844 0.03532497]
  ...
  [0.51661763 0.10988873]
  [0.7387359  0.78058581]
  [0.20141776 0.57190349]]

 [[0.7216099  0.63182755]
  [0.29711514 0.93615345]
  [0.2004391  0.48064726]
  ...
  [0.55503896 0.40408063]
  [0.52101464 0.94862893]
  [0.27673919 0.67572063]]]


In [5]:
#converting the state space from box format to discrete format. Used to input into our q learing algorithm.
def get_discrete_state(state):
    discrete_state = state/step_size+ np.array([15,10,1,10])
    return tuple(discrete_state.astype(np.int))

In [ ]:
#iterating through n epochs
for epoch in range(epochs + 1): 
    #set the initial time, so we can calculate how much each action takes
    t_initial = time.time() 
    
    #get the discrete state for the restarted environment
    discrete_state = get_discrete_state(env.reset()) 
    
    #we create a boolean that will tell us whether our game is running or not
    done = False
    
    #our reward is intialized at zero at the beginning of every eisode
    epoch_reward = 0 

    #Every 1000 epochs we have an episode. Measuring reward for every set of epochs.
    if epoch % 1000 == 0: 
        print("Episode: " + str(epoch)) 

    while not done: 
        #Now we are in our gameloop
        #if some random number is greater than epsilon, then we take the best possible action (max reward) we have explored so far
        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state])
        
        #else, we will explore and take a random action
        else:

            action = np.random.randint(0, env.action_space.n) 

        #now we will intialize our new_state, reward, and done variables
        new_state, reward, done, _ = env.step(action) 
    
        epoch_reward += reward 
        
        #we discretize our new state
        new_discrete_state = get_discrete_state(new_state)
        
        #we render our environment after 2000 steps
        if epoch % 2000 == 0: 
            env.render()

        #if the game loop is still running, update the q-table
        if not done:
            max_new_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - lr) * current_q + lr * (reward + gamma* max_new_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state
    # if our epsilon is greater than .05m , and if our reward is greater than the previous and if we reached past our 10000 epoch, we recalculate episilon
    
    if epsilon > 0.05: 
        if epoch_reward > prev_reward and epoch > 10000:
            epsilon = math.pow(epsilon_decay_value, epoch - 10000)

           

    #we calculate the final time
    tfinal = time.time() 
    
    #total epoch time
    episode_total = tfinal - t_initial 
    total_time += episode_total
    
    #calculate and update rewards
    total_reward += epoch_reward
    prev_reward = epoch_reward

    #every 1000 episodes print the average time and the average reward
    if epoch % 1000 == 0: 
        mean_time = total_time / 1000
        print("Time Average: " + str(mean_time))
        total_time = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

        print("Epsilon: " + str(epsilon))
   
env.close()


Episode: 0


c:\users\linda\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


Time Average: 0.0011647839546203612
Mean Reward: 0.015
Epsilon: 1
Episode: 1000
Time Average: 0.002135960102081299
Mean Reward: 22.487
Epsilon: 1
Episode: 2000
Time Average: 0.0026190588474273684
Mean Reward: 21.339
Epsilon: 1
Episode: 3000
Time Average: 0.0021013717651367186
Mean Reward: 22.275
Epsilon: 1
Episode: 4000
Time Average: 0.002555295705795288
Mean Reward: 22.863
Epsilon: 1
Episode: 5000
Time Average: 0.002084465742111206
Mean Reward: 21.897
Epsilon: 1
Episode: 6000
Time Average: 0.0023908872604370118
Mean Reward: 22.001
Epsilon: 1
Episode: 7000
Time Average: 0.002145289182662964
Mean Reward: 22.802
Epsilon: 1
Episode: 8000
Time Average: 0.002377060890197754
Mean Reward: 22.944
Epsilon: 1
Episode: 9000
Time Average: 0.002109410047531128
Mean Reward: 22.68
Epsilon: 1
Episode: 10000
Time Average: 0.0022804863452911377
Mean Reward: 22.097
Epsilon: 1
Episode: 11000
Time Average: 0.0021313350200653077
Mean Reward: 22.597
Epsilon: 0.9512757992150065
Episode: 12000
Time Average: 0.